In [ ]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from IPython.display import display  # ใช้สำหรับแสดง DataFrame ใน Jupyter Notebook

class FuzzyKNN:
    def __init__(self, k=5, m=2):
        """
        Fuzzy K-Nearest Neighbors classifier.

        Parameters:
        - k: จำนวนเพื่อนบ้าน (ต้องเป็นเลขคี่)
        - m: Fuzziness parameter (ค่าความ fuzzy)
        """
        self.k = k
        self.m = m

    def predict(self, train_labels, test_labels, distance_matrix):
        """
        ทำนายข้อมูล Test โดยใช้ค่าระยะห่างที่คำนวณไว้แล้ว

        Parameters:
        - train_labels: อาร์เรย์ของ label สำหรับข้อมูล train (200 แถว)
        - test_labels: อาร์เรย์ของ label สำหรับข้อมูล test (500 แถว)
        - distance_matrix: เมทริกซ์ระยะห่าง (200x500)

        Returns:
        - DataFrame แสดงผลลัพธ์ Fuzzy Membership ของแต่ละ test sample
        """
        N, M = distance_matrix.shape
        assert len(train_labels) == N, f"train_labels ต้องมีขนาด {N}"
        assert len(test_labels) == M, f"test_labels ต้องมีขนาด {M}"

        predictions = []

        for j in range(M):  # วนลูปทุกแถวของ Test Data
            distances = distance_matrix[:, j]  # ดึงระยะห่างของ Test ตัวที่ j กับทุก Train
            sorted_indices = np.argsort(distances)[:self.k]  # เลือก K ตัวที่ใกล้ที่สุด
            nearest_labels = train_labels[sorted_indices]
            nearest_distances = distances[sorted_indices]

            # คำนวณสมาชิก (Fuzzy Membership)
            membership = defaultdict(float)
            if np.any(nearest_distances == 0):  # ถ้ามีตัวที่ตรงกันพอดี
                label = nearest_labels[np.argmin(nearest_distances)]
                predictions.append({label: 1.0})
                continue

            inv_distances = 1 / (nearest_distances ** (2 / (self.m - 1)))
            total_inv = np.sum(inv_distances)

            for i, label in enumerate(nearest_labels):
                membership[label] += inv_distances[i] / total_inv

            predictions.append(membership)

        # แปลง Fuzzy Membership เป็น Label ที่ทำนายได้
        predicted_labels = [max(membership, key=membership.get) for membership in predictions]

        # สร้าง DataFrame แสดงผล
        df_results = pd.DataFrame({
            "Test Label": test_labels, 
            "Predicted Label": predicted_labels
        })

        return df_results

# 🟢 **โหลดไฟล์และทดสอบค่าที่ดีที่สุด**
if __name__ == "__main__":
    # 📌 **กำหนดพาธไฟล์**
    train_label_path = "C:/Users/BMEI CMU/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_CYBER_SECURITY/RESULT/05.DATA_VALIDATION/fold_1/MALWARE_100_BENIGN_100/validation_train.json"
    test_label_path = "C:/Users/BMEI CMU/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_CYBER_SECURITY/RESULT/05.DATA_VALIDATION/fold_1/MALWARE_100_BENIGN_100/validation_test.json"
    distance_matrix_path = "C:/Users/BMEI CMU/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_CYBER_SECURITY/RESULT/06.EDIT_DISTANCE_VALIDATION/fold_1/MATRIX_EDIT_DISTANCE_MALWARE_100_BENIGN_100.csv"

    # 📌 **โหลด Labels จาก JSON**
    with open(train_label_path, "r", encoding="utf-8") as file:
        train_labels_data = json.load(file)
    
    with open(test_label_path, "r", encoding="utf-8") as file:
        test_labels_data = json.load(file)

    # 📌 **แปลง JSON เป็น DataFrame**
    train_labels_df = pd.DataFrame(train_labels_data)
    test_labels_df = pd.DataFrame(test_labels_data)

    # 📌 **ดึงค่า Label จาก Column 2 (Index 1)**
    train_labels = train_labels_df.iloc[:, 1].to_numpy()  # Column 2
    test_labels = test_labels_df.iloc[:, 1].to_numpy()  # Column 2

    # 📌 **โหลด Distance Matrix (200x500)**
    distance_matrix_df = pd.read_csv(distance_matrix_path, header=None)
    distance_matrix = distance_matrix_df.to_numpy()

    # 📌 **รันค่าต่างๆ ของ k และ m เพื่อหา Accuracy ที่ดีที่สุด**
    k_min, k_max = 1, 21  # k เริ่มที่ 1 และเพิ่มทีละ 2 (1, 3, 5, ..., 21)
    m_min, m_max = 2, 5   # m เริ่มที่ 2 และเพิ่มทีละ 1 (2, 3, 4, 5)

    best_accuracy = 0
    best_k = None
    best_m = None

    for k in range(k_min, k_max + 1, 2):  # k เป็นเลขคี่เท่านั้น
        for m in np.arange(m_min, m_max + 0.1, 0.1):  # m วิ่งตั้งแต่ 2 ถึง m_max
            # สร้างและ Train Model
            fuzzy_knn = FuzzyKNN(k=k, m=m)
            df_results = fuzzy_knn.predict(train_labels, test_labels, distance_matrix)

            # 📌 **คำนวณ Accuracy**
            correct_predictions = (df_results["Test Label"] == df_results["Predicted Label"]).sum()
            total_predictions = len(df_results)
            accuracy = (correct_predictions / total_predictions) * 100

            print(f"🔹 k={k}, m={m} → Accuracy: {accuracy:.2f}% ({correct_predictions}/{total_predictions} ถูกต้อง)")

            # อัปเดตค่าที่ดีที่สุด
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_k = k
                best_m = m

    # 📌 **แสดงค่าที่ดีที่สุด**
    print(f"\n✅ ค่า k และ m ที่ให้ Accuracy สูงสุด:")
    print(f"🔹 Best k = {best_k}, Best m = {best_m}, Best Accuracy = {best_accuracy:.2f}%")


TypeError: 'float' object cannot be interpreted as an integer